In [32]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

# Track data loading
import psutil
import time

# Save&Load results
import pickle

# 1. Import Dataset

In [2]:
file_path = 'Data/covid19_infodemic_RT_en_1584482894_1686211361_filtered_sorted.dat'

num_rows = sum(1 for line in open(file_path))

In [3]:
num_rows 

114161884

- **Header Data**

Header data is stored in this dataset in the final row.

In [4]:
data_header = np.loadtxt(file_path, dtype = 'str', delimiter = '|', skiprows = num_rows - 1, max_rows = 1)

In [5]:
data_header

array(['tweetID', 'tweetTimestamp', 'tweetType', 'fromUserID',
       'fromUserIsBot', 'fromUserIsVerified', 'fromUserFollowers',
       'fromUserFriends', 'toUserID', 'toUserIsBot', 'toUserIsVerified',
       'toUserFollowers', 'toUserFriends', 'newsCategory'], dtype='<U18')

We manually clarify content in each column sequentially (14 columns in total):

- **Col 0 - 2**: 'tweetID', 'tweetTimestamp', 'tweetType';
- **Col 3 - 7**: 'fromUserID', 'fromUserIsBot', 'fromUserIsVerified', 'fromUserFollowers', 'fromUserFriends'; 
- **Col 8 - 12**: 'toUserID', 'toUserIsBot', 'toUserIsVerified', 'toUserFollowers', 'toUserVerified';
- **Col 13**: 'newsCategory'

Meanwhile, we skip this ending row when loading the dataset in the following.

- **Data Sample Line**

In [6]:
data_line_sample = np.loadtxt(file_path, dtype = 'str', delimiter = '|', max_rows = 1)

In [7]:
data_line_sample

array(['1240052352247197699', '1584482894.0', 'RT', 'HmmBee', '0', '0',
       '286', '571', 'jacobinmag', '1', '1', '263225', '63', 'POLITICAL'],
      dtype='<U19')

We assign data separately to variables 'data_str' and 'data_int' to avoid excessive RAM consumption associated with importing the entire dataset as 'string' type.

- **data_str**: tweetType, fromUserID, toUserID, newsCategory

In [8]:
st = time.time()

data_str = np.loadtxt(file_path, max_rows = num_rows - 1, dtype = 'str', delimiter = '|', usecols = (2, 3, 8, 13))

et = time.time()
elapsed_time = et - st

print('Execution time:', elapsed_time/60, 'mins')
# Getting % usage of virtual_memory ( 3rd field)
print('RAM memory % used:', psutil.virtual_memory()[2])
# Getting usage of virtual_memory in GB ( 4th field)
print('RAM Used (GB):', psutil.virtual_memory()[3]/1000000000)

Execution time: 3.004303153355916 mins
RAM memory % used: 64.1
RAM Used (GB): 255.764033536


In [9]:
np.shape(data_str)

(114161883, 4)

- **data_int**: 'tweetID', 'tweetTimestamp', 'fromUserIsBot', 'fromUserIsVerified', 'fromUserFollowers', 'fromUserFriends', 'toUserIsBot', 'toUserIsVerified', 'toUserFollowers', 'toUserFriends'

In [10]:
st = time.time()

data_int = np.loadtxt(file_path, max_rows = num_rows - 1, dtype = np.int64, delimiter = "|", usecols = (0,1,4,5,6,7,9,10,11,12))

et = time.time()
elapsed_time = et - st

print('Execution time:', elapsed_time/60, 'mins')
# Getting % usage of virtual_memory ( 3rd field)
print('RAM memory % used:', psutil.virtual_memory()[2])
# Getting usage of virtual_memory in GB ( 4th field)
print('RAM Used (GB):', psutil.virtual_memory()[3]/1000000000)

Execution time: 1.5282966335614523 mins
RAM memory % used: 66.3
RAM Used (GB): 264.738480128


In [11]:
np.shape(data_int)

(114161883, 10)

# 2. Data Cleaning

- **repeated tweetIDs**: Remove data lines with repeated tweetIDs (0.25%)

In [12]:
index_retain1 = np.unique(data_int[:, 0], return_index=True)[1]

In [13]:
len(data_int[:, 0]) - len(index_retain1), 1 - len(index_retain1)/len(data_int[:, 0])

(296078, 0.0025934926108392986)

In [14]:
index_retain1

array([        0,         1,         2, ..., 114161880, 114161881,
       114161882])

- **newsCategory**: there are 189,355 data lines (0.16%) with newsCateogry "" (i.e., empty). "NA" refers to retweets that contain URLs but not included in our URLs database. 

In [15]:
len(np.where(data_str[:, -1] == "")[0]), len(np.where(data_str[:, -1] == "")[0])/len(data_str[:, -1])

(189355, 0.001658653440395688)

In [18]:
index_remove2 = np.where(data_str[:, -1] == "")[0]

In [19]:
len(index_remove2), len(index_remove2)/len(data_str[:, -1])

(189355, 0.001658653440395688)

In [20]:
all_index = np.arange(0, len(data_str[:, -1]), 1)
mask = np.isin(all_index, index_remove2)
index_retain2 = all_index[~mask]

In [21]:
index_retain2

array([        0,         1,         2, ..., 114161880, 114161881,
       114161882])

- **fromUserFollowers, fromUserFriends**: there is a single data line where both fromUserFollowers and fromUserFriends are recorded as -1, corresponding to tweetID 1445440938491396098

In [22]:
index_remove3 = np.where(data_int[:, 0] == 1445440938491396098)[0]

In [23]:
len(index_remove3), len(index_remove3)/len(data_str[:, -1])

(1, 8.759491116662818e-09)

In [24]:
mask = np.isin(all_index, index_remove3)
index_retain3 = all_index[~mask]

In [25]:
index_retain3

array([        0,         1,         2, ..., 114161880, 114161881,
       114161882])

**Find Intersection**

In [26]:
index_retain = np.intersect1d(index_retain1, np.intersect1d(index_retain2, index_retain3))

In [27]:
index_retain

array([        0,         1,         2, ..., 114161880, 114161881,
       114161882])

In [28]:
len(index_retain), len(index_retain)/len(all_index)

(113685100, 0.9958236235469241)

## 2.1. Save&Load Retained Index

- Save

In [87]:
with open('Data/dataCleaning_indexRetain.pickle', 'wb') as file:
    pickle.dump(index_retain, file)

In [40]:
# Export DataFrame to Excel

df = pd.DataFrame(data_int[:, 0][index_retain], columns=['tweetID'])
df.to_csv('Data/tweetID_clean.csv', index=False)

- Load

In [18]:
with open('Data/dataCleaning_indexRetain.pickle', 'rb') as file:
    index_retain =  pickle.load(file)

In [30]:
with open('Data/tweetID_clean.pickle', 'rb') as file:
    tweetID_clean = pickle.load(file)

In [31]:
tweetID_clean

array([1240052352247197699, 1240052352326877184, 1240052352993972225, ...,
       1624634217161269248, 1624634243052716032, 1624634245334421504])

# 3. SummaryStats

## 3.1. Data Timeframe

Start from 7 March 2020, collected second by second

In [19]:
min(data_int[:, 1][index_retain]), max(data_int[:, 1][index_retain])

(1584482894, 1676174362)

In [20]:
(max(data_int[:, 1][index_retain]) - min(data_int[:, 1][index_retain]))/(24*60*60)

1061.2438425925925

In [21]:
from datetime import datetime, timedelta

# Starting date
start_date = datetime(2020, 3, 7)

# Calculate the date 1061 days later
result_date = start_date + timedelta(days=1061)

print("Date 1061 days after March 7, 2020 is:", result_date.strftime("%Y-%m-%d"))

Date 1061 days after March 7, 2020 is: 2023-02-01


## 3.2. News Category

In [22]:
newsCategory = np.unique(data_str[:, -1][index_retain])
newsCategory

array(['CLICKBAIT', 'CONSPIRACY/JUNKSCI', 'FAKE/HOAX', 'MSM', 'NA',
       'OTHER', 'POLITICAL', 'SATIRE', 'SCIENCE', 'SHADOW'], dtype='<U18')

In [24]:
category_counts

{'CLICKBAIT': 1926902,
 'CONSPIRACY/JUNKSCI': 1783052,
 'FAKE/HOAX': 5088717,
 'MSM': 40711848,
 'NA': 29615585,
 'OTHER': 2793839,
 'POLITICAL': 12202886,
 'SATIRE': 258135,
 'SCIENCE': 1183951,
 'SHADOW': 18120185}

- scientific

In [25]:
sci_sum = category_counts['SCIENCE'] + category_counts['MSM']

In [26]:
sci_sum, sci_sum/len(data_str[:, -1][index_retain])

(41895799, 0.3685249782073464)

- fake

In [27]:
fake_sum = category_counts['CONSPIRACY/JUNKSCI'] + category_counts['FAKE/HOAX'] + category_counts['CLICKBAIT']

In [28]:
fake_sum, fake_sum/len(data_str[:, -1][index_retain])

(8798671, 0.07739511158454362)

- uncertain

In [29]:
uncertain_sum = len(data_str[:, -1][index_retain]) - sci_sum - fake_sum

In [30]:
uncertain_sum, uncertain_sum/len(data_str[:, -1][index_retain])

(62990630, 0.55407991020811)

### 3.2.1. Encoding & Decoding

- Encode tweetNewsCategory into integers (so as to reduce computation time)

In [31]:
newsCateogry_dict_encode = {newsCategory[i]:i for i in range(len(newsCategory))}

In [32]:
newsCateogry_dict_encode

{'CLICKBAIT': 0,
 'CONSPIRACY/JUNKSCI': 1,
 'FAKE/HOAX': 2,
 'MSM': 3,
 'NA': 4,
 'OTHER': 5,
 'POLITICAL': 6,
 'SATIRE': 7,
 'SCIENCE': 8,
 'SHADOW': 9}

- Decode

In [33]:
newsCateogry_dict_decode = {i:newsCategory[i] for i in range(len(newsCategory))}

In [34]:
newsCateogry_dict_decode

{0: 'CLICKBAIT',
 1: 'CONSPIRACY/JUNKSCI',
 2: 'FAKE/HOAX',
 3: 'MSM',
 4: 'NA',
 5: 'OTHER',
 6: 'POLITICAL',
 7: 'SATIRE',
 8: 'SCIENCE',
 9: 'SHADOW'}

In [37]:
newsCateogry_dict_generalised_code = {3: 0,  # factual
                                      8: 0,
                                      0: 1,  # misleading
                                      1: 1,
                                      2: 1,
                                      4: 2,  # undecided
                                      5: 2,
                                      6: 2, 
                                      7: 2,
                                      9: 2,
                                     }

- Save & Load

In [38]:
with open('Graphs/newsCateogry_dict_decode.pickle', 'wb') as file:
    pickle.dump(newsCateogry_dict_decode, file)
with open('Graphs/newsCateogry_dict_generalised_code.pickle', 'wb') as file:
    pickle.dump(newsCateogry_dict_generalised_code, file)

In [39]:
with open('Graphs/newsCateogry_dict_decode.pickle', 'rb') as file:
    newsCateogry_dict_decode =  pickle.load(file)

## 3.3. Users

In [37]:
unique_users = np.union1d(data_str[:, 1][index_retain], data_str[:, 2][index_retain])

In [38]:
len(unique_users)

13948159

### 3.3.1. Encoding & Decoding

- Encode users' ID into integers (so as to reduce computation time)

In [41]:
user_dict_encode = {unique_users[i]:i for i in range(len(unique_users))}

- Decode

In [47]:
user_dict_decode = {i:unique_users[i] for i in range(len(unique_users))}

- Save & Load

In [51]:
with open('Graphs/user_dict_decode.pickle', 'wb') as file:
    pickle.dump(user_dict_decode, file)

In [52]:
with open('Graphs/user_dict_decode.pickle', 'rb') as file:
    user_dict_decode =  pickle.load(file)

In [53]:
user_dict_decode[10645250], user_dict_decode[6827149] # example

('ladbible', 'WhiteHouse')